# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('Orders.zip')
orders = orders.drop('Unnamed: 0', axis = 1)
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [12]:
# First things first: let's look for missing values and duplicate values.
print('Shape of dataframe before droping duplicates:', orders.shape)
print('')
orders.drop_duplicates()
print('Shape of dataframe after droping duplicates:', orders.shape)
print('')
print(orders.info())

Shape of dataframe before droping duplicates: (397924, 13)

Shape of dataframe after droping duplicates: (397924, 13)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   InvoiceNo     397924 non-null  int64  
 1   StockCode     397924 non-null  object 
 2   year          397924 non-null  int64  
 3   month         397924 non-null  int64  
 4   day           397924 non-null  int64  
 5   hour          397924 non-null  int64  
 6   Description   397924 non-null  object 
 7   Quantity      397924 non-null  int64  
 8   InvoiceDate   397924 non-null  object 
 9   UnitPrice     397924 non-null  float64
 10  CustomerID    397924 non-null  int64  
 11  Country       397924 non-null  object 
 12  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(7), object(4)
memory usage: 39.5+ MB
None


#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

*First, we need to group the orders by customer ids.*

In [17]:
print('How many unique customer ids are there?', 
      orders['CustomerID'].nunique())

How many unique customer ids are there? 4339


*Let's group our data and check if the rows we're getting as a result of this operation coincide with the previous value.*

In [18]:
gunique_customers = orders.groupby('CustomerID').agg({'amount_spent':sum})
print(unique_customers.shape)
unique_customers.head()

(4339, 1)


,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


#### Sub Problem 2&3: How to select customers whose aggregated `amount_spent` is in a given quantile range? / How to label selected customers as "VIP" or "Preferred"?

*By using the quantile function, we can get the exact value from which we can discriminate if a specific value is or is not in a given quantile range.*

*This method also allows us to apply a filter to our data and select those values which are in the specified desired range*

In [24]:
vip_amount = unique_customers.quantile(0.95)
print('Minimum amount for vip customers: {:.2f}'.format(vip_amount[0]))
vip_customers = unique_customers[unique_customers['amount_spent'] > vip_amount[0]]
display(vip_customers.describe().T)
vip_customers.head()

Minimum amount for vip customers: 5840.18


,count,mean,std,min,25%,50%,75%,max
amount_spent,217.0,20688.48023,35073.151824,5870.08,7044.68,9736.52,16587.09,280206.02


,amount_spent
CustomerID,
12346,77183.60
12357,6207.67
12359,6372.58
12409,11072.67
12415,124914.53


In [25]:
preferred_amount = unique_customers.quantile([0.75, 0.95])
display('Min and max amount of preferred customers:', preferred_amount)
pref_customers = unique_customers[(unique_customers['amount_spent'] >= 1661.640) & (unique_customers['amount_spent'] <= 5840.182)]
display(pref_customers.describe().T)
pref_customers.head()

'Min and max amount of preferred customers:'

,amount_spent
0.75,1661.640
0.95,5840.182


,count,mean,std,min,25%,50%,75%,max
amount_spent,868.0,2951.780462,1071.278907,1661.84,2058.81,2639.055,3646.3475,5836.86


,amount_spent
CustomerID,
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
12356,2811.43


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

*By grouping our data by Customer_ID and Country, we can get the country that concentrates most of the VIP Customers. First, we'll store all the index values for each row of VIP and preferred customers. Then, we can group them by the two mentioned variables, and finally filter by customer ID.*

In [26]:
selected_customer_ids = list(vip_customers.index)
vip_country = orders.loc[orders['CustomerID'].isin(selected_customer_ids)]
vip_grouped = vip_country.groupby(['CustomerID','Country']).agg({'amount_spent':sum}).reset_index()
vip_grouped['Country'].value_counts()

United Kingdom     177
Germany             10
France               9
Switzerland          3
Australia            2
Spain                2
Japan                2
Portugal             2
EIRE                 2
Finland              1
Belgium              1
Norway               1
Denmark              1
Cyprus               1
Singapore            1
Netherlands          1
Channel Islands      1
Sweden               1
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [29]:
vip_pref_ids = list(vip_customers.index)+list(pref_customers.index)
vip_pref_country = orders.loc[orders['CustomerID'].isin(vip_pref_ids)]
vip_pref_grouped = vip_pref_country.groupby(['CustomerID','Country']).agg({'amount_spent':sum}).reset_index()
vip_pref_grouped['Country'].value_counts()

United Kingdom     932
Germany             39
France              29
Belgium             12
Switzerland          9
Spain                9
Portugal             7
Norway               7
Italy                5
Finland              5
Japan                4
Cyprus               4
Australia            4
Channel Islands      4
Austria              3
Denmark              3
EIRE                 3
Sweden               2
Israel               2
Singapore            1
Lebanon              1
Poland               1
Greece               1
Netherlands          1
Iceland              1
Canada               1
Malta                1
Name: Country, dtype: int64

*In both cases, United Kingdom is the country with the higher number of VIP and VIP+Preferred customers.*